# Redis私房手册

## 安装

### windows安装

官网只有Linux版本下载，windows到 https://github.com/MSOpenTech/redis/releases 下载。  
可视化管理软件Redis Desktop Manager0.94版本以后就收费了，百度盘里保存了0.93版本。

### Centos安装

## 配置和命令行工具

### windows下的配置

安装以后自动在windows的后台作为一个服务运行，默认的端口为6397。配置文件是安装目录下的`redis.windows-service.conf`文件，注意：还有一个`redis.windows.conf`，需要修改的是`redis.windows-service.conf`文件而不是`redis.windows.conf`。几个常见的配置如下：
- 修改密码：`requirepass <password>`
- 设置最大内存：`maxmemory <bytes>`，注意可以用简写，比如限制100m，可以写成`maxmemory 100mb`。单位格式配置文档一开头就有写。

修改以后要重启服务。很奇怪的是，隔天开机以后服务就消失了。只好又手工配置，手工启动有两种方式：
1. 把`redis`作为一个程序运行，可以把它理解成一个服务器。
    - 在`dos`界面输入`redis-server.exe d:\programs\redis\redis.windows-service.conf`，注意如果没有配置环境变量的话，`redis-server.exe`前面要加路径。或者进入`redis`的安装目录，可以不用路径。此时界面会被阻塞，意思是光标会卡着不动，表示服务器开始运行，此时`redis`可以登录。也可以不加配置文件，直接`redis-server.exe`，此时会以默认配置启动，但是发现这个默认配置即不是`redis.windows.conf`，也不是`redis.windows-service.conf`。


2. 把`redis`作为一个服务来运行，可以理解成一个后台程序，可以随着`windows`启动而启动。
    - 在`dos`界面输入`redis-server.exe --service-install d:\programs\redis\redis.windows-service.conf`，等一下会提示安装完成，安装完成以后可以在windows的服务列表里面看到的，然后再用`redis-server.exe --service-start`启动服务，使用`redis-server.exe --service-stop`关闭服务。删除服务可以使用`redis-server.exe --service-uninstall`命令。注意，这种方式是非阻塞的，运行了命令不会卡着不动。

没有搞明白的是，配置文件可以自己指定，使用`redis.windows.conf`或者`redis.windows-service.conf`都可以，如果不指定，则使用默认配置，而默认配置这两者都不是，因此没明白这两个文件有什么区别。

### 命令行工具

在`dos`界面下运行`redis-cli.exe`可以进入`redis`的命令行界面，前提是`redis`作为服务器或者服务已经运行起来了，这里记录几个常用的命令：
- `info`：查看`redis`的信息，包括cpu，内存什么的占用。
- `auth`：**<font color='red'>注意：</font>**不管有没有设置密码，`redis-cli`都可以登录，这个和`mysql`一开始就需要输入密码登录不一样。但是设置了密码的话，登录以后不能进行任何操作，需要再输入`auth`命令，输入密码以后才行。
- `config get requirepass`：获取当前数据库密码。

输入命令开头的字母，使用`tab`键可以切换命令。

## Redis数据类型

### 有序集合 

[这篇文章比较全面](https://www.cnblogs.com/huchong/p/9656882.html#_label6)，这里只列举学习过程中遇到的容易出错的知识点：

#### 添加元素`zadd`

`redis3.3.8`版本下，按一些书上写法`rcdb.zadd(key, score, name)`添加新的元素报错，正确的应该这样`rcdb.zadd(key, {name:score})`，完整的函数签名是：`zadd(name, mapping, nx=False, xx=False, ch=False, incr=False)`，其中：
- `nx`：只新增`name`，`name`存在，`score`不同，不会更新`score`。
- `xx`：只更新`score`，`name`存在，`score`不同，则更新`score`，`name`不存在，也不会插入新的`name`。
- `ch`：修改返回结果为发生变化的元素数量，默认情况是返回新增`name`数量。
- `incr`：这个模式下,`mapping`只能是单个的`{name：amount}`对，注意此时是`amount`，即`score`的增量，返回的也是新的`score`。

注意默认行为是(`nx`和`xx`都是`False`)，如果`name`存在，`score`不同，更新`score`，如果`name`不存在，则插入新的`name`。

#### 查看`name`是否存在`zscore` 

注意，是查看`name`而不是`key`是否存在，查看`key`是否存在，可以直接用`exists`方法。`zscore`返回`name`的`score`，如果不存在，则返回`None`。

#### 选取方法

主要有`zrange`,`zrevrange`，`zrangebyscore`，`zrevrangebyscore`，`zrangebylex`，`zrevrangebylex`。可以这样理解：
- 所有方法先按照`score`值从小到大排序，不包含`rev`是前往后选取，包含`rev`是从后往前选取。
- 只有`range`是按照位置，`byscore`是按照分数，`bylex`是按照`name`值的字典顺序，语法有点奇怪，[点这里](https://blog.csdn.net/qq_37058039/article/details/80108014)。
- `zrange`有个`desc`参数，设置为`True`以后就是从大到小，和`zrevrange`一样。
- `bylex`主要是针对`score`分数一样的情况，如果不一样，可能会返回不正确的结果，另外，参数`min`和`max`要加上`[`或者`(`，分别表示包含和不包含`，也可以用`+`和`-`代替，注意：`python`里不能用`+`和`-`代替。

#### 返回排名

`zrank`和`zrevrank`先按照`score`从小到大排序，然后分别返回从前到后和从后到前的`name`值的排名。

## API文档

- [中文官方教程](https://www.redis.net.cn/order/)。
- [API 命令](https://redis.io/commands)查询。
- `python`的`redis`库增加了[几个指令](https://pypi.org/project/redis/)

## 常见问题收集

### `redis`里插入数据后再读出来时所有键与值都是`byte`类型的，怎么办？

连接`redis`的时候，有个参数为`decode_responses`，默认为`False`，设置为`True`即可自动转换。